# Data Exploration in R
This notebook demonstrates basic data exploration for futures data using R.

In [ ]:
# Load required libraries
library(tidyverse)
library(arrow)  # For reading parquet files

In [ ]:
# Read a sample parquet batch
df <- read_parquet('data/processed/batches/ZNZF_1m_batch1.parquet')
head(df)

In [ ]:
# Summary statistics
summary(df)

In [ ]:
# Plot close prices for each symbol
df_long <- df %>% pivot_longer(-ts_event, names_to = 'symbol', values_to = 'close')
ggplot(df_long, aes(x = ts_event, y = close, color = symbol)) + geom_line() + theme_minimal()